In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import numpy as np
import pandas as pd
import joblib

import os
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt


import pathlib
DATA_DIR = pathlib.Path.cwd()/'data/input'
OUT_DIR = pathlib.Path.cwd()/'data/output'

import sys 
sys.path.append(str(pathlib.Path.cwd()/'utils'))
from utils.misc_utils import fullrange, realized_volatility, log_return, rmspe, get_stock_path, load_parquet_file, load_parquet_files, load_train_test
from utils.feature_engineering_utils import full_feature_engineering, groupby_and_aggregate, generate_features_book_data, generate_features_trade_data, full_feature_engineering_by_cutoff

book_aggregation = {
    'wap1': [np.mean, np.std, fullrange], 
    'wap2': [np.mean, np.std, fullrange], 
    'log_return_1': [fullrange, np.sum, np.mean, realized_volatility], 
    'log_return_2': [fullrange, np.sum, np.mean, realized_volatility], 
    'bid_ask_price_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_price_spread_2': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_1': [np.mean, np.std, fullrange],
    'bid_ask_size_spread_2': [np.mean, np.std, fullrange]
    }

trade_aggregation = {
    'volume': [np.mean, np.sum, np.std], 
    'price': [np.mean, np.std], 
    'order_count': [np.mean, np.sum, np.std]
    }

time_agg_trade = {
    'volume_mean': [np.mean, np.sum, np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.sum, np.std]
    }

time_agg_book = {
    'wap1_std': [np.mean], 
    'wap2_std': [np.mean], 
    'log_return_1_realized_volatility': [np.mean], 
    'log_return_2_realized_volatility': [np.mean], 
    'log_return_1_sum': [np.mean], 
    'log_return_2_sum': [np.mean] 
    }

stock_agg_trade = {
    'volume_mean': [np.mean,np.std], 
    'price_mean': [np.mean, np.std], 
    'order_count_mean': [np.mean, np.std]
    }

stock_agg_book = {
    'wap1_std': [np.std], 
    'wap2_std': [np.std], 
    'log_return_1_realized_volatility': [np.std], 
    'log_return_2_realized_volatility': [np.std], 
    'log_return_1_sum': [np.std], 
    'log_return_2_sum': [np.std] 
    }

pd.set_option('display.max_columns', None)

In [3]:
training_target = pd.read_csv(DATA_DIR/'train.csv')
book = load_parquet_files(stock_ids=[0, 126], file_type='book')
trade = load_parquet_files(stock_ids=[0, 126], file_type='trade')

100%|██████████| 2/2 [00:00<00:00,  5.93it/s]


In [4]:
book = generate_features_book_data(book)
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,id,wap1,wap2,log_return_1,log_return_2,bid_ask_price_spread_1,bid_ask_price_spread_2,bid_ask_size_spread_1,bid_ask_size_spread_2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0,0-5,1.001434,1.001390,0.000000,0.000000,0.000879,0.000983,223,98
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0,0-5,1.001448,1.001390,0.000014,0.000000,0.000879,0.000983,97,98
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0,0-5,1.001448,1.001391,0.000000,0.000001,0.000879,0.001034,97,98
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,0-5,1.001443,1.001391,-0.000005,0.000000,0.000879,0.001034,123,98
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,0-5,1.001443,1.001391,0.000000,0.000000,0.000879,0.001034,123,98


In [5]:
agg_book_data = groupby_and_aggregate(book, agg_col = 'id', agg_dict=book_aggregation)
agg_book_data.head()

,wap1_mean,wap1_std,wap1_fullrange,wap2_mean,wap2_std,wap2_fullrange,log_return_1_fullrange,log_return_1_sum,log_return_1_mean,log_return_1_realized_volatility,log_return_2_fullrange,log_return_2_sum,log_return_2_mean,log_return_2_realized_volatility,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange
id,,,,,,,,,,,,,,,,,,,,,,,,,,
0-1000,0.998800,0.000531,0.001829,0.998738,0.000538,0.002868,0.001286,-0.000944,-0.000006,0.001781,0.002234,-0.001198,-0.000007,0.004389,0.000443,0.000173,0.000878,0.000765,0.000249,0.001478,11.695122,171.482507,1116,28.609756,83.794630,599
0-10000,0.999884,0.000395,0.001409,0.999912,0.000488,0.002080,0.001733,0.000546,0.000002,0.002890,0.001641,0.000686,0.000002,0.004111,0.000460,0.000232,0.001022,0.000739,0.000239,0.001314,10.143357,152.601616,1045,-17.912587,130.656572,747
0-10005,1.001301,0.001809,0.007130,1.001160,0.001676,0.007296,0.004685,0.002516,0.000014,0.008674,0.006666,0.000928,0.000005,0.013725,0.002107,0.000602,0.003449,0.002743,0.000602,0.003025,-11.918478,103.390706,532,28.766304,107.369872,694
0-10017,0.996141,0.004763,0.017300,0.996030,0.004678,0.018040,0.016086,-0.002971,-0.000013,0.017629,0.018510,-0.003512,-0.000015,0.021224,0.003538,0.001323,0.007071,0.004959,0.002062,0.010697,60.934211,158.634259,860,-0.675439,151.850436,700
0-10030,0.999464,0.000433,0.001729,0.999477,0.000498,0.002206,0.001503,0.000580,0.000003,0.002551,0.004305,0.003158,0.000016,0.005463,0.000623,0.000246,0.001085,0.001040,0.000305,0.001463,33.577320,83.867072,364,6.876289,85.745528,416


In [6]:
trade = generate_features_trade_data(trade)
trade.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,id,volume
0,5,21,1.002301,326,12,0,0-5,326.750244
1,5,46,1.002778,128,4,0,0-5,128.355591
2,5,50,1.002818,55,1,0,0-5,55.155014
3,5,57,1.003155,121,5,0,0-5,121.381798
4,5,68,1.003646,4,1,0,0-5,4.014584


In [7]:
agg_trade_data =  groupby_and_aggregate(trade, agg_col = 'id', agg_dict=trade_aggregation)
agg_trade_data.head()

,volume_mean,volume_sum,volume_std,price_mean,price_std,order_count_mean,order_count_sum,order_count_std
id,,,,,,,,
0-1000,29.961796,898.853882,45.738499,0.998908,0.000554,2.133333,64,2.029665
0-10000,80.825371,1939.808960,132.549789,0.999931,0.000373,2.916667,70,2.500725
0-10005,81.590820,2366.133789,119.623756,1.001232,0.001756,3.137931,91,3.502286
0-10017,89.547050,3760.976074,133.678314,0.996374,0.004358,3.166667,133,3.875606
0-10030,98.030350,2940.910645,103.179214,0.999314,0.000493,2.333333,70,1.667816


In [8]:
output_df = agg_book_data.merge(agg_trade_data, left_index=True, right_index=True).reset_index()
output_df['stock_id'] = output_df['id'].apply(lambda x: int(x.split('-')[0]))
output_df['time_id'] = output_df['id'].apply(lambda x: int(x.split('-')[1]))
output_df.head()

,id,wap1_mean,wap1_std,wap1_fullrange,wap2_mean,wap2_std,wap2_fullrange,log_return_1_fullrange,log_return_1_sum,log_return_1_mean,log_return_1_realized_volatility,log_return_2_fullrange,log_return_2_sum,log_return_2_mean,log_return_2_realized_volatility,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean,volume_sum,volume_std,price_mean,price_std,order_count_mean,order_count_sum,order_count_std,stock_id,time_id
0,0-1000,0.998800,0.000531,0.001829,0.998738,0.000538,0.002868,0.001286,-0.000944,-0.000006,0.001781,0.002234,-0.001198,-0.000007,0.004389,0.000443,0.000173,0.000878,0.000765,0.000249,0.001478,11.695122,171.482507,1116,28.609756,83.794630,599,29.961796,898.853882,45.738499,0.998908,0.000554,2.133333,64,2.029665,0,1000
1,0-10000,0.999884,0.000395,0.001409,0.999912,0.000488,0.002080,0.001733,0.000546,0.000002,0.002890,0.001641,0.000686,0.000002,0.004111,0.000460,0.000232,0.001022,0.000739,0.000239,0.001314,10.143357,152.601616,1045,-17.912587,130.656572,747,80.825371,1939.808960,132.549789,0.999931,0.000373,2.916667,70,2.500725,0,10000
2,0-10005,1.001301,0.001809,0.007130,1.001160,0.001676,0.007296,0.004685,0.002516,0.000014,0.008674,0.006666,0.000928,0.000005,0.013725,0.002107,0.000602,0.003449,0.002743,0.000602,0.003025,-11.918478,103.390706,532,28.766304,107.369872,694,81.590820,2366.133789,119.623756,1.001232,0.001756,3.137931,91,3.502286,0,10005
3,0-10017,0.996141,0.004763,0.017300,0.996030,0.004678,0.018040,0.016086,-0.002971,-0.000013,0.017629,0.018510,-0.003512,-0.000015,0.021224,0.003538,0.001323,0.007071,0.004959,0.002062,0.010697,60.934211,158.634259,860,-0.675439,151.850436,700,89.547050,3760.976074,133.678314,0.996374,0.004358,3.166667,133,3.875606,0,10017
4,0-10030,0.999464,0.000433,0.001729,0.999477,0.000498,0.002206,0.001503,0.000580,0.000003,0.002551,0.004305,0.003158,0.000016,0.005463,0.000623,0.000246,0.001085,0.001040,0.000305,0.001463,33.577320,83.867072,364,6.876289,85.745528,416,98.030350,2940.910645,103.179214,0.999314,0.000493,2.333333,70,1.667816,0,10030


## Time Period Aggregation

In [9]:
time_agg_trade_data = groupby_and_aggregate(output_df, agg_col = 'time_id', agg_dict=time_agg_trade, suffix='_period')
time_agg_trade_data.head()

,volume_mean_mean_period,volume_mean_sum_period,volume_mean_std_period,price_mean_mean_period,price_mean_std_period,order_count_mean_mean_period,order_count_mean_sum_period,order_count_mean_std_period
time_id,,,,,,,,
5,63.489216,126.978432,23.001141,1.002913,0.001144,2.669118,5.338235,0.114385
11,75.123787,150.247574,45.460377,1.000473,0.000377,2.709259,5.418519,1.144465
16,109.012894,218.025787,32.058147,0.999548,0.000486,2.744615,5.489231,0.034811
31,81.574921,163.149841,69.389015,0.998861,0.000225,3.050000,6.100000,1.249222
62,67.259079,134.518158,19.963213,0.999277,0.000483,3.320025,6.640049,1.025913


In [10]:
time_agg_book_data = groupby_and_aggregate(output_df, agg_col = 'time_id', agg_dict=time_agg_book, suffix='_period')
time_agg_book_data.head()

,wap1_std_mean_period,wap2_std_mean_period,log_return_1_realized_volatility_mean_period,log_return_2_realized_volatility_mean_period,log_return_1_sum_mean_period,log_return_2_sum_mean_period
time_id,,,,,,
5,0.000876,0.000990,0.006064,0.009029,0.002922,0.002750
11,0.000503,0.000533,0.003823,0.004540,-0.002060,-0.001627
16,0.000802,0.000840,0.002699,0.005010,-0.002588,-0.002912
31,0.000809,0.000708,0.004144,0.004617,-0.000918,-0.000466
62,0.000413,0.000478,0.003319,0.004168,0.001852,0.001475


In [11]:
time_agg = time_agg_book_data.merge(time_agg_trade_data, left_index=True, right_index=True)
output_df_with_time = output_df.merge(time_agg, on='time_id')
output_df_with_time = output_df_with_time.merge(training_target, on=['stock_id', 'time_id']).head()

In [12]:
train, test = load_train_test()
train_id = train.stock_id.unique()
test_id = test.stock_id.unique()

In [13]:
#final_training_data = full_feature_engineering(stock_ids = [], train=True, training_target= training_target)
#final_test_data = full_feature_engineering(stock_ids = [], train=False)

cutoffs =  [(0,150), (150, 300), (300, 450), (450, 600)]
final_training_data, datalist_train = full_feature_engineering_by_cutoff(
    cutoffs = cutoffs,
    stock_ids = train_id,
    train=True)

final_test_data, datalist_test = full_feature_engineering_by_cutoff(
    cutoffs = cutoffs,
    stock_ids = test_id,
    train=False)

#final_training_data = train.merge(final_training_data, on = ['id', 'stock_id', 'time_id'], how='left')
#final_test_data = test.merge(final_test_data, on = ['id', 'stock_id', 'time_id'], how='left')

final_training_data.to_pickle(OUT_DIR/'final_training_data.pkl')
final_test_data.to_pickle(OUT_DIR/'final_test_data.pkl')


100%|██████████| 1/1 [00:00<00:00, 22.07it/s]


In [ ]:
[len(dataset) for dataset in datalist_train]

In [ ]:
final_training_data.head()

,id,time_id,stock_id,wap1_mean_0_300,wap1_std_0_300,wap1_fullrange_0_300,wap2_mean_0_300,wap2_std_0_300,wap2_fullrange_0_300,log_return_1_fullrange_0_300,log_return_1_sum_0_300,log_return_1_mean_0_300,log_return_1_realized_volatility_0_300,log_return_2_fullrange_0_300,log_return_2_sum_0_300,log_return_2_mean_0_300,log_return_2_realized_volatility_0_300,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean_0_300,volume_sum_0_300,volume_std_0_300,price_mean_0_300,price_std_0_300,order_count_mean_0_300,order_count_sum_0_300,order_count_std_0_300,wap1_std_mean_period_0_300,wap2_std_mean_period_0_300,log_return_1_realized_volatility_mean_period_0_300,log_return_2_realized_volatility_mean_period_0_300,log_return_1_sum_mean_period_0_300,log_return_2_sum_mean_period_0_300,volume_mean_mean_period_0_300,volume_mean_sum_period_0_300,volume_mean_std_period_0_300,price_mean_mean_period_0_300,price_mean_std_period_0_300,order_count_mean_mean_period_0_300,order_count_mean_sum_period_0_300,order_count_mean_std_period_0_300,wap1_std_std_stock_0_300,wap2_std_std_stock_0_300,log_return_1_realized_volatility_std_stock_0_300,log_return_2_realized_volatility_std_stock_0_300,log_return_1_sum_std_stock_0_300,log_return_2_sum_std_stock_0_300,volume_mean_mean_stock_0_300,volume_mean_std_stock_0_300,price_mean_mean_stock_0_300,price_mean_std_stock_0_300,order_count_mean_mean_stock_0_300,order_count_mean_std_stock_0_300,wap1_mean_300_600,wap1_std_300_600,wap1_fullrange_300_600,wap2_mean_300_600,wap2_std_300_600,wap2_fullrange_300_600,log_return_1_fullrange_300_600,log_return_1_sum_300_600,log_return_1_mean_300_600,log_return_1_realized_volatility_300_600,log_return_2_fullrange_300_600,log_return_2_sum_300_600,log_return_2_mean_300_600,log_return_2_realized_volatility_300_600,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean_300_600,volume_sum_300_600,volume_std_300_600,price_mean_300_600,price_std_300_600,order_count_mean_300_600,order_count_sum_300_600,order_count_std_300_600,wap1_std_mean_period_300_600,wap2_std_mean_period_300_600,log_return_1_realized_volatility_mean_period_300_600,log_return_2_realized_volatility_mean_period_300_600,log_return_1_sum_mean_period_300_600,log_return_2_sum_mean_period_300_600,volume_mean_mean_period_300_600,volume_mean_sum_period_300_600,volume_mean_std_period_300_600,price_mean_mean_period_300_600,price_mean_std_period_300_600,order_count_mean_mean_period_300_600,order_count_mean_sum_period_300_600,order_count_mean_std_period_300_600,wap1_std_std_stock_300_600,wap2_std_std_stock_300_600,log_return_1_realized_volatility_std_stock_300_600,log_return_2_realized_volatility_std_stock_300_600,log_return_1_sum_std_stock_300_600,log_return_2_sum_std_stock_300_600,volume_mean_mean_stock_300_600,volume_mean_std_stock_300_600,price_mean_mean_stock_300_600,price_mean_std_stock_300_600,order_count_mean_mean_stock_300_600,order_count_mean_std_stock_300_600,target
0,0-1000,1000,0,0.999208,0.000427,0.001128,0.999107,0.000398,0.001455,0.000991,-0.001171,-0.000015,0.001184,0.001793,-0.001089,-0.000014,0.002684,0.000422,0.000136,0.000601,0.000744,0.000202,0.000785,58.772152,129.514572,482,42.506329,91.762496,599,26.356178,421.698853,37.846413,0.999351,0.000284,2.125000,34.0,1.408309,0.000367,0.000402,0.001448,0.002109,-0.000198,-0.000207,334.647034,37480.468750,1207.417847,1.000150,0.000896,4.118892,461.315938,2.652600,0.000716,0.000787,

In [ ]:
final_training_data.isna().sum()

id                                     0
time_id                                0
stock_id                               0
wap1_mean_0_300                        0
wap1_std_0_300                         0
                                      ..
price_mean_mean_stock_300_600          0
price_mean_std_stock_300_600           0
order_count_mean_mean_stock_300_600    0
order_count_mean_std_stock_300_600     0
target                                 0
Length: 124, dtype: int64

## Model Training

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram, plot_convergence

rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

In [ ]:
final_training_data = pd.read_pickle(OUT_DIR/'final_training_data.pkl')
final_test_data = pd.read_pickle(OUT_DIR/'final_test_data.pkl')
#final_training_data = final_training_data.dropna(axis=1)

In [ ]:
from sklearn.utils import resample

model_col = [col for col in final_training_data.columns if ('id' not in col) & ('target' not in col)]

X_train, X_test, y_train, y_test = train_test_split(
                                        final_training_data.drop('target', axis=1)[model_col],
                                        final_training_data['target'],
                                        test_size=0.1
                                        )

X_train_sample = resample(X_train, replace=False, n_samples=5000)
y_train_sample = y_train.loc[X_train_sample.index]

In [ ]:
final_training_data

,id,time_id,stock_id,wap1_mean_0_300,wap1_std_0_300,wap1_fullrange_0_300,wap2_mean_0_300,wap2_std_0_300,wap2_fullrange_0_300,log_return_1_fullrange_0_300,log_return_1_sum_0_300,log_return_1_mean_0_300,log_return_1_realized_volatility_0_300,log_return_2_fullrange_0_300,log_return_2_sum_0_300,log_return_2_mean_0_300,log_return_2_realized_volatility_0_300,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean_0_300,volume_sum_0_300,volume_std_0_300,price_mean_0_300,price_std_0_300,order_count_mean_0_300,order_count_sum_0_300,order_count_std_0_300,wap1_std_mean_period_0_300,wap2_std_mean_period_0_300,log_return_1_realized_volatility_mean_period_0_300,log_return_2_realized_volatility_mean_period_0_300,log_return_1_sum_mean_period_0_300,log_return_2_sum_mean_period_0_300,volume_mean_mean_period_0_300,volume_mean_sum_period_0_300,volume_mean_std_period_0_300,price_mean_mean_period_0_300,price_mean_std_period_0_300,order_count_mean_mean_period_0_300,order_count_mean_sum_period_0_300,order_count_mean_std_period_0_300,wap1_std_std_stock_0_300,wap2_std_std_stock_0_300,log_return_1_realized_volatility_std_stock_0_300,log_return_2_realized_volatility_std_stock_0_300,log_return_1_sum_std_stock_0_300,log_return_2_sum_std_stock_0_300,volume_mean_mean_stock_0_300,volume_mean_std_stock_0_300,price_mean_mean_stock_0_300,price_mean_std_stock_0_300,order_count_mean_mean_stock_0_300,order_count_mean_std_stock_0_300,wap1_mean_300_600,wap1_std_300_600,wap1_fullrange_300_600,wap2_mean_300_600,wap2_std_300_600,wap2_fullrange_300_600,log_return_1_fullrange_300_600,log_return_1_sum_300_600,log_return_1_mean_300_600,log_return_1_realized_volatility_300_600,log_return_2_fullrange_300_600,log_return_2_sum_300_600,log_return_2_mean_300_600,log_return_2_realized_volatility_300_600,bid_ask_price_spread_1_mean,bid_ask_price_spread_1_std,bid_ask_price_spread_1_fullrange,bid_ask_price_spread_2_mean,bid_ask_price_spread_2_std,bid_ask_price_spread_2_fullrange,bid_ask_size_spread_1_mean,bid_ask_size_spread_1_std,bid_ask_size_spread_1_fullrange,bid_ask_size_spread_2_mean,bid_ask_size_spread_2_std,bid_ask_size_spread_2_fullrange,volume_mean_300_600,volume_sum_300_600,volume_std_300_600,price_mean_300_600,price_std_300_600,order_count_mean_300_600,order_count_sum_300_600,order_count_std_300_600,wap1_std_mean_period_300_600,wap2_std_mean_period_300_600,log_return_1_realized_volatility_mean_period_300_600,log_return_2_realized_volatility_mean_period_300_600,log_return_1_sum_mean_period_300_600,log_return_2_sum_mean_period_300_600,volume_mean_mean_period_300_600,volume_mean_sum_period_300_600,volume_mean_std_period_300_600,price_mean_mean_period_300_600,price_mean_std_period_300_600,order_count_mean_mean_period_300_600,order_count_mean_sum_period_300_600,order_count_mean_std_period_300_600,wap1_std_std_stock_300_600,wap2_std_std_stock_300_600,log_return_1_realized_volatility_std_stock_300_600,log_return_2_realized_volatility_std_stock_300_600,log_return_1_sum_std_stock_300_600,log_return_2_sum_std_stock_300_600,volume_mean_mean_stock_300_600,volume_mean_std_stock_300_600,price_mean_mean_stock_300_600,price_mean_std_stock_300_600,order_count_mean_mean_stock_300_600,order_count_mean_std_stock_300_600,target
0,0-1000,1000,0,0.999208,0.000427,0.001128,0.999107,0.000398,0.001455,0.000991,-0.001171,-0.000015,0.001184,0.001793,-0.001089,-0.000014,0.002684,0.000422,0.000136,0.000601,0.000744,0.000202,0.000785,58.772152,129.514572,482,42.506329,91.762496,599,26.356178,421.698853,37.846413,0.999351,0.000284,2.125000,34.0,1.408309,0.000367,0.000402,0.001448,0.002109,-0.000198,-0.000207,334.647034,37480.468750,1207.417847,1.000150,0.000896,4.118892,461.315938,2.652600,0.000716,0.000787,

In [ ]:
model_pipeline = Pipeline(
    steps=[
        ('scaler',   StandardScaler()),
        ('xgboost',   XGBRegressor())
        ]
    )

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
rmspe(y_test, xgb.predict(X_test))

0.23809433938933214

In [ ]:
xg_grid = {'learning_rate':Real(0.01, 0.4, prior='uniform'),
        'gamma': Real(0, 10, prior='uniform'),
        'reg_alpha':  Real(0, 10, prior='uniform'),
        'colsample_bytree': Real(0.6, 1, prior='uniform'),
        'max_depth':  Integer(5,18), 
        'n_estimators':  Integer(5,250)
        }

cv_xg = BayesSearchCV(
    XGBRegressor(),
    xg_grid,
    n_iter=100,
    random_state=123, 
    verbose=0, 
    cv=3, 
    refit=True, 
    return_train_score=True
    )

cv_xg.fit(X_train, y_train)
joblib.dump(cv_xg, OUT_DIR/'models/xgboost.pkl')

KeyboardInterrupt: 

In [ ]:
rmspe(y_test, cv_xg.best_estimator_.predict(X_test))

In [ ]:
rmspe(y_test, xgb.predict(X_test))

In [ ]:
cv_xg = joblib.load(OUT_DIR/'models/xgboost.pkl')

In [ ]:
cv_xg.best_params_

In [ ]:
xg_tuned = XGBRegressor(
    gamma = 0, 
    learning_rate = 0.148727, 
    max_depth = 12,
    n_estimators = 191,
    reg_alpha = 3.53674,
    subsample = 1
    )

#xg_tuned = XGBRegressor(**cv_xg.best_params_)
xg_tuned.fit(X_train, y_train)

In [ ]:
rmspe(y_train, xg_tuned.predict(X_train))

In [ ]:
rmspe(y_test, xg_tuned.predict(X_test))

In [ ]:
submission = final_test_data[['id']].rename(columns = {'id': 'row_id'})
submission['target'] = cv_xg.best_estimator_.predict(final_test_data[model_col])

In [ ]:
submission.info()

In [ ]:
submission = final_test_data[['id']].rename(columns = {'id': 'row_id'})
submission['target'] = xg_tuned.predict(final_test_data[model_col])
submission.to_csv('submission.csv',index = False)

In [ ]:
submission_sample = pd.read_csv(DATA_DIR/'sample_submission.csv')
submission_sample.info()